# Paquete: LifeContingencies

El paquete `lifecontingencies` tiene muchas funciones relacionadas con cálculo actuarial general, y un enfoque en introducir una estrutura de datos conveniente para el manejo de tablas de vida como las vistas en clases teóricas.

## Funciones básicas

Tenemos una función que relaciona directamente los factores de de descuento, $d$, con los intereses efectivos, $i$ según la siguiente equivalencia:
$$
(1+i)^t = (1-d)^{-t}
$$
Cargamos la librería:

In [1]:
# install.package("lifecontingencies")
library("lifecontingencies")

Package:  lifecontingencies
Authors:  Giorgio Alfredo Spedicato [aut, cre]
    (<https://orcid.org/0000-0002-0315-8888>),
  Christophe Dutang [ctb] (<https://orcid.org/0000-0001-6732-1501>),
  Reinhold Kainhofer [ctb] (<https://orcid.org/0000-0002-7895-1311>),
  Kevin J Owens [ctb],
  Ernesto Schirmacher [ctb],
  Gian Paolo Clemente [ctb] (<https://orcid.org/0000-0001-6795-4595>),
  Ivan Williams [ctb]
Version:  1.3.6
Date:     2019-03-05 23:50:03 UTC
BugReport: http://github.com/spedygiorgio/lifecontingencies/issues




Pasaremos de un interés efectivo (en la unidad de tiempo que sea), $i = 0.03$, hacia un factor de descuento, $d$, en las mismas unidades:

In [2]:
interest2Discount(0.03)

[1] 0.02912621

Podemos también calcularlo de vuelta:

In [3]:
discount2Interest(interest2Discount(0.03))

[1] 0.03

Podemos también transformar un interés convertible (según algún periodo de tiempo) hacia uno efectivo, en la misma unidad de tiempo. Es decir, bajo la relación:
$$
(1 + i)^t = \left(1 + \frac{i^{(k)}}{k} \right)^t
$$

In [4]:
convertible2Effective(i=0.10, k=4)

[1] 0.1038129

## Valores presentes y tasa interna de retorno

Considere el siguiente flujo de dinero en los siguientes tiempo:

In [5]:
capitales <- c(-1000,200,500,700)
tiempos <- c(0,1,2,5)

Podemos calcular el valor presente de ese flujo de dinero mediante la siguiente función

In [6]:
presentValue(cashFlows=capitales, timeIds=tiempos, interestRates=0.03)

[1] 269.2989

Podemos también considerar tasas de intereses variables a lo largo del lapso que cubre el flujo:

In [7]:
intereses <- c(0.04, 0.02, 0.03, 0.05)
presentValue(cashFlows=capitales, timeIds=tiempos, interestRates=intereses)

[1] 215.8447

Esto es sumamente útil cuando los vectores `capitales` y `tiempos` sean considerados continuos (muchos valores en tiempos pequeños respecto a la longitud del lapso) y el interés se estime de variar acorde a una función $i(t)$, representado en `intereses`. 

Además. Se sabe que en la realidad los valores del flujo de dinero pueden no ser siempre seguros y tener cierto riesgo asociado y modelado aquí por una probabilidad de obtenerlo o no obtenerlo. Esto es posible también:

In [8]:
probabilidades <- c(1,0.7,1,0.9)
presentValue(cashFlows=capitales, timeIds=tiempos, 
             interestRates=intereses, probabilities=probabilidades)

[1] 102.1743

En este caso el vector `probabilidades` puede contener una distribución de probabilidad. Por ejemplo:

In [9]:
probabilidades <- pnorm(rnorm(4))
probabilidades
presentValue(cashFlows=capitales, timeIds=tiempos, 
             interestRates=intereses, probabilities=probabilidades)

[1] 0.3482378 0.8157142 0.6645904 0.8028481

[1] 565.263

La tasa interna de retorno (IRR) es la tasa de interés que resulta en un valor presente de exactamente cero, manteniendo fijas las demás variables (los valores del flujo, la probabilidad, etc)

In [10]:
# Nos creamos una función igual al VP al cuadrado por motivos de optimización
getIRR <- function(p) (presentValue(cashFlows=capitales,
                                    timeIds=tiempos, interestRates=p))^2
nlm(f=getIRR, p=0.1)$estimate

[1] 0.1105091

In [11]:
str(nlm(f=getIRR, p=0.1))

List of 5
 $ minimum   : num 1.9e-06
 $ estimate  : num 0.111
 $ gradient  : num 0.00704
 $ code      : int 2
 $ iterations: int 4


In [12]:
?nlm

nlm {stats},R Documentation
f,"the function to be minimized, returning a single numeric value. This should be a function with first argument a vector of the length of p followed by any other arguments specified by the ... argument. If the function value has an attribute called gradient or both gradient and hessian attributes, these will be used in the calculation of updated parameter values. Otherwise, numerical derivatives are used. deriv returns a function with suitable gradient attribute and optionally a hessian attribute."
p,starting parameter values for the minimization.
...,additional arguments to be passed to f.
hessian,"if TRUE, the hessian of f at the minimum is returned."
typsize,an estimate of the size of each parameter at the minimum.
fscale,an estimate of the size of f at the minimum.
print.level,"this argument determines the level of printing which is done during the minimization process. The default value of 0 means that no printing occurs, a value of 1 means that initial and final details are printed and a value of 2 means that full tracing information is printed."
ndigit,the number of significant digits in the function f.
gradtol,a positive scalar giving the tolerance at which the scaled gradient is considered close enough to zero to terminate the algorithm. The scaled gradient is a measure of the relative change in f in each direction p[i] divided by the relative change in p[i].
stepmax,"a positive scalar which gives the maximum allowable scaled step length. stepmax is used to prevent steps which would cause the optimization function to overflow, to prevent the algorithm from leaving the area of interest in parameter space, or to detect divergence in the algorithm. stepmax would be chosen small enough to prevent the first two of these occurrences, but should be larger than any anticipated reasonable step."


## Anualidades y valores acumulados futuros

Considere una anualidad de 5 pagos con montos de Lps. 1200 y un interés efectivo de $i=0.03$

In [19]:
1200 * annuity(i=0.03, n=5)

[1] 5495.649

Podemos también saber su valor acumulado:

In [20]:
1200 * accumulatedValue(i=0.03, n=5)

[1] 6370.963

Es posible que los pagos de la anualidad se hagan en 5 periodos (digamos, años) como en este ejemplo, pero se paguen con cierta frecuencia $k$ a lo largo de ese año (pagos de $ \frac{1200}{k}$ cada uno)

In [21]:
1200 * annuity(i=0.03, n=5, k=12)

[1] 5570.811

Para el caso en que la anualidad no se vence, y por ende es una perpetuidad, podemos calcularlo así

In [22]:
1200 * annuity(i=0.03, n=Inf)

[1] 40000

... Aunque esto pudo ser igual fácilmente calculado mediante:

In [23]:
1200/0.03

[1] 40000

Pero la idea es centralizar todos los cálculos en una sola función que pueda dinámica y automáticamente responder a diversas situaciones.

Por último. El cálculo de la función `annuity` es, por defecto, asumiendo que la anualidad es de pagos vencidos. Para obtener pagos anticipados se puede utilizar el argumento `type`

In [24]:
1200 * annuity(i=0.03, n=5, type="advance")

[1] 5660.518

## Tablas de vida

In [25]:
x_example <- seq(from=0,to=9, by=1)
lx_example <- c(1000,950,850,700,680,600,550,400,200,50)
exampleLt <- new("lifetable", x=x_example, lx=lx_example, name="example lifetable")

In [26]:
str(exampleLt)

Formal class 'lifetable' [package "lifecontingencies"] with 3 slots
  ..@ x   : num [1:10] 0 1 2 3 4 5 6 7 8 9
  ..@ lx  : num [1:10] 1000 950 850 700 680 600 550 400 200 50
  ..@ name: chr "example lifetable"


In [27]:
exampleLt

Life table example lifetable 

  x   lx        px       ex
1 0 1000 0.9500000 4.980000
2 1  950 0.8947368 4.242105
3 2  850 0.8235294 3.741176
4 3  700 0.9714286 3.542857
5 4  680 0.8823529 2.647059
6 5  600 0.9166667 2.000000
7 6  550 0.7272727 1.181818
8 7  400 0.5000000 0.625000
9 8  200 0.2500000 0.250000


El paquete `lifecontingencies` provee ya algunos ejemplos de datos de tasas de natalidad y mortalidad. Por ejemplo consideremos:

In [28]:
data("demoUsa")

In [29]:
head(demoUsa)

,age,USSS2007M,USSS2007F,USSS2000M,USSS2000F,USSS1990M,USSS1990F
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,100000,100000,100000,100000,100000,100000
2,1,99262,99390,99241,99377,98972,99185
3,2,99213,99347,99187,99332,98896,99120
4,3,99182,99322,99150,99302,98844,99083
5,4,99158,99303,99122,99283,98804,99053
6,5,99138,99288,99100,99264,98771,99028


In [30]:
usaMale07 <- demoUsa[,c("age", "USSS2007M")]
usaMale00 <- demoUsa[,c("age", "USSS2000M")]

Renombramos por estética

In [32]:
names(usaMale07) <- c("x","lx")
names(usaMale00) <- c("x","lx")

In [33]:
head(usaMale07)

,x,lx
,<int>,<dbl>
1,0,100000
2,1,99262
3,2,99213
4,3,99182
5,4,99158
6,5,99138


In [34]:
tabla <- as(usaMale07,"lifetable")
tabla

Life table COERCED 

      x     lx        px        ex
1     0 100000 0.9926200 74.881620
2     1  99262 0.9995064 74.438355
3     2  99213 0.9996875 73.475119
4     3  99182 0.9997580 72.498084
5     4  99158 0.9997983 71.515632
6     5  99138 0.9998184 70.530059
7     6  99120 0.9998386 69.542867
8     7  99104 0.9998486 68.554095
9     8  99089 0.9998587 67.564472
10    9  99075 0.9998991 66.574020
11   10  99065 0.9999092 65.580740
12   11  99056 0.9999091 64.586698
13   12  99047 0.9998486 63.592567
14   13  99032 0.9997476 62.602199
15   14  99007 0.9996061 61.618007
16   15  98968 0.9994342 60.642288
17   16  98912 0.9992822 59.676622
18   17  98841 0.9991198 58.719489
19   18  98754 0.9989874 57.771219
20   19  98654 0.9988546 56.829779
21   20  98541 0.9987112 55.894947
22   21  98414 0.9985876 54.967078
23   22  98275 0.9985042 54.044823
24   23  98128 0.9984918 53.125785
25   24  97980 0.9985099 52.206032
26   25  97834 0.9985588 51.283940
27   26  97693 0.9985874 50.357958

In [35]:
getOmega(tabla)

[1] 111

In [36]:
exampleAct <- new("actuarialtable",x=tabla@x, lx=tabla@lx, 
                  interest=0.03, name="Tabla Actuarial")

In [37]:
print(exampleAct)

Actuarial table  Tabla Actuarial interest rate  3 % 

      x     lx           Dx           Nx           Cx           Mx           Rx
1     0 100000 1.000000e+05 3.000556e+06 716.50485437 1.260516e+04 7.864341e+05
2     1  99262 9.637087e+04 2.900556e+06  46.18719955 1.188865e+04 7.738290e+05
3     2  99213 9.351777e+04 2.804185e+06  28.36939144 1.184247e+04 7.619403e+05
4     3  99182 9.076558e+04 2.710668e+06  21.32368915 1.181410e+04 7.500979e+05
5     4  99158 8.810060e+04 2.619902e+06  17.25217569 1.179277e+04 7.382838e+05
6     5  99138 8.551731e+04 2.531801e+06  15.07471662 1.177552e+04 7.264910e+05
7     6  99120 8.301144e+04 2.446284e+06  13.00946418 1.176045e+04 7.147155e+05
8     7  99104 8.058062e+04 2.363273e+06  11.84113851 1.174744e+04 7.029550e+05
9     8  99089 7.822177e+04 2.282692e+06  10.72983425 1.173560e+04 6.912076e+05
10    9  99075 7.593274e+04 2.204470e+06   7.44093915 1.172487e+04 6.794720e+05
11   10  99065 7.371366e+04 2.128538e+06   6.50179149 1.171742e+04